In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import rasterio
import numpy as np

# Load the training data
file_path = r'C:\...\Plot_Extract_Data+Estimated_MH.csv' #Path to file with training data
training_data = pd.read_csv(file_path)
training_data = training_data[training_data['UniqueID'] != 1846]

# Define predictors and the target variable
predictor_names = [
    'B3_Jul', 'B3_Jun', 'B5_Jul', 'B5_Jun', 'CAB_May', 'GLCM_VV_dent_Jan', 
    'MCARI_Aug', 'Predicted_Height_A', 'TCARI_Jul', 'TCARI_Jun', 'sarRI2_Feb', 'sarSqDI_Sep' ]  #Best predictor set from optimisation algorithm

X = training_data[predictor_names]
y = training_data['Dw_All_Mg'] #Target variable

# Initialize and train the RandomForest Regressor
rf = RandomForestRegressor(n_estimators=100, max_features=10, random_state=7)  #Best hyperparamters from optimisation algorithm
rf.fit(X, y)

def predict_by_window(src, model, predictor_descriptions):
    """Predict raster data by window, using dynamic band indexing based on descriptions."""
    for ji, window in src.block_windows(1):
        print(f"Processing window {ji}...")
        data = []
        valid_data_mask = np.ones((window.height, window.width), dtype=bool)
        
        # Dynamically read bands based on descriptions
        for desc in predictor_descriptions:
            band_index = src.indexes[src.descriptions.index(desc)]
            band_data = src.read(band_index, window=window)
            data.append(band_data)
            valid_data_mask &= np.isfinite(band_data)

        data = np.array(data)
        flat_data = data.reshape(len(predictor_descriptions), -1).T
        flat_valid_mask = valid_data_mask.ravel()
        df_predict = pd.DataFrame(flat_data[flat_valid_mask], columns=predictor_descriptions)

        result = np.full(flat_valid_mask.shape, np.nan, dtype=np.float32)
        if not df_predict.empty:
            result[flat_valid_mask] = model.predict(df_predict)
        yield window, result.reshape(window.height, window.width)

# Load the raster image and setup output
input_raster = r'C:\...\raster.tif' #Set path to raster
output_path = r'C:\...\Estimated_AGB.tif' #Set output location and file name

with rasterio.open(input_raster) as src:
    profile = src.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw')

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.set_band_description(1, 'Estimated_AGB')  # Setting the ouput band description
        for window, result in predict_by_window(src, rf, predictor_names):
            dst.write(result.astype(rasterio.float32), 1, window=window)


Processing window (0, 0)...
Processing window (1, 0)...
Processing window (2, 0)...
Processing window (3, 0)...
Processing window (4, 0)...
Processing window (5, 0)...
Processing window (6, 0)...
Processing window (7, 0)...
Processing window (8, 0)...
Processing window (9, 0)...
Processing window (10, 0)...
Processing window (11, 0)...
Processing window (12, 0)...
Processing window (13, 0)...
Processing window (14, 0)...
Processing window (15, 0)...
Processing window (16, 0)...
Processing window (17, 0)...
Processing window (18, 0)...
Processing window (19, 0)...
Processing window (20, 0)...
Processing window (21, 0)...
Processing window (22, 0)...
Processing window (23, 0)...
Processing window (24, 0)...
Processing window (25, 0)...
Processing window (26, 0)...
Processing window (27, 0)...
Processing window (28, 0)...
Processing window (29, 0)...
Processing window (30, 0)...
Processing window (31, 0)...
Processing window (32, 0)...
Processing window (33, 0)...
Processing window (34, 0

In [2]:
# Extract feature importances from the model
feature_importances = rf.feature_importances_

# Create a DataFrame for easier viewing
importances_df = pd.DataFrame({
    'Feature': predictor_names,
    'Importance': feature_importances
})

# Sort the DataFrame by importance in descending order
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Display the importances
print(importances_df)

# Save to CSV
importances_df.to_csv(r'C:\...\predictor_importance.csv', index=False) #Set path to ouput table

               Feature  Importance
7   Predicted_Height_A    0.544296
6            MCARI_Aug    0.057647
4              CAB_May    0.057512
3               B5_Jun    0.052619
5     GLCM_VV_dent_Jan    0.046667
2               B5_Jul    0.046120
11         sarSqDI_Sep    0.044098
1               B3_Jun    0.038724
9            TCARI_Jun    0.035315
10          sarRI2_Feb    0.026726
8            TCARI_Jul    0.025381
0               B3_Jul    0.024895


In [3]:
#Calculate raster stats
def calculate_agb_statistics(agb_raster_path):
    """
    Calculate the minimum, maximum, and average values of an AGB raster.

    Parameters:
    agb_raster_path (str): The file path to the AGB raster file.

    Returns:
    tuple: A tuple containing the minimum, maximum, and average AGB values.
    """
    with rasterio.open(agb_raster_path) as src:
        agb_data = src.read(1)  # Read the first band

        # Mask out no-data values, assuming they are represented as NaNs
        agb_masked = np.ma.masked_invalid(agb_data)

        # Calculate statistics
        min_agb = agb_masked.min()
        max_agb = agb_masked.max()
        avg_agb = agb_masked.mean()

    return min_agb, max_agb, avg_agb

# Path to your AGB raster file
agb_raster_path = r'C:\...\Estimated_AGB.tif'

# Calculate and print the AGB statistics
min_agb, max_agb, avg_agb = calculate_agb_statistics(agb_raster_path)
print(f"Minimum AGB value: {min_agb}")
print(f"Maximum AGB value: {max_agb}")
print(f"Average AGB value: {avg_agb}")


Minimum AGB value: 36.95806121826172
Maximum AGB value: 367.4366455078125
Average AGB value: 253.7302557081007
